<div style="background-color:#7a1400;height:60px;padding-top:7px;">
<h3 align=center style="color:white">
Multivariate Sales Forecasting - German Retail Chain - Multivariate Forecasting
</h3> 
</div>

<h4>Avinash Pasupulate</h4><br>
avinash.pasupulate@gmail.com<br>
<a href="www.avinash.pasupulate.com">www.avinash.pasupulate.com</a>
<br><br>
<i>7th January 2019</i>

<br>
<hr>
<b>Open the html file in the repository or click 
<a href="http://nbviewer.jupyter.org/github/avinashpasupulate/multivariate_sales_forecasting_german_retail_chain/blob/master/sales_forecasting.ipynb">Nbconvert Notebook</a> to view plots that are properly rendered.
<br><br>
Please wait for the html page to load completely.</b>
<br>
<hr>

<h3>Methods for Multivariate Forecasting: </h3>
<hr>
<ul>
    <li> Linear Regression
    <li> ARIMAX - Auto-Regressive Integrated Moving Average with Exogenous variables
    <li> SARIMAX - Seasonal Auto-Regressive Integrated Moving Average with Exogenous variables
    <li> VAR - Vector AutoRegression
    <li> Prophet by Facebook
     </ul>
<br><br>       
 
 
<h3>Steps to Complete: </h3>
<hr>
<ul>
    <li> Clean-up & remove/replace null values

In [143]:
import os
import copy
import warnings
import random #setting seed

#data 
import pandas as pd

#visualization
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm._tqdm_notebook import tqdm_notebook

#data pre-processing
from sklearn.preprocessing import OneHotEncoder


% matplotlib inline
tqdm_notebook.pandas()
warnings.filterwarnings('ignore')
pd.set_option('float_format','{:f}'.format)

In [144]:
cwd = os.getcwd()
random.seed(9891482)

In [145]:
forecast_ads = pd.read_csv(cwd+r'/data/processed_data/sales_ads.csv')

<h3>Data Cleanup</h3>
<hr>
<ul>
    <li> Check for missing and partial values
    <li> Replace / correct missing or partial values
    <li> Remove rows containing NA values
    <li> Final Checks
</ul>

In [156]:
#making copy of the imported data
forecast_clean = copy.deepcopy(forecast_ads)

In [157]:
#checking for missing and partial information
num = forecast_clean.describe(include='all').T
num['count']=num['count'].astype('int64')
num[['count', 'unique']].sort_values('count', ascending=True)

count unique
CompetitionDistance     2611969    NaN
Unnamed: 0              2617643    NaN
days_since_competition  2617643   9461
Assortment              2617643      3
StoreType               2617643      4
relh                    2617643    NaN
tmpc                    2617643    NaN
cpi                     2617643    NaN
num_asy_app             2617643     31
wc_days                 2617643    NaN
google_svi              2617643    NaN
mapped_region           2617643     12
quarter                 2617643    NaN
days_since_promo2       2617643   2183
seasons                 2617643      4
weekday                 2617643      7
iso_weeknum             2617643    NaN
day                     2617643    NaN
year                    2617643    NaN
SchoolHoliday           2617643    NaN
StateHoliday            2617643      4
Promo                   2617643    NaN
Open                    2617643    NaN
Customers               2617643    NaN
Sales                   2617643    NaN
date_s                  2617643    942
Store                   2617643    NaN
month                   2617643     12
promo2_int_mod          2617643    NaN

In [158]:
num[['count', 'unique']][num['unique']<10]

count unique
StateHoliday  2617643      4
weekday       2617643      7
seasons       2617643      4
StoreType     2617643      4
Assortment    2617643      3

In [159]:
#replacing na values in the CompetitionDistance column with 0 
forecast_clean.loc[forecast_clean['CompetitionDistance'].isna(),
                   'CompetitionDistance']=0

In [160]:
#replace timedelta values with numbers in days_since_competition and days_since_promo2 columns
forecast_clean['days_since_competition'] = pd.to_timedelta(forecast_clean['days_since_competition']).dt.days
forecast_clean['days_since_promo2'] = pd.to_timedelta(forecast_clean['days_since_promo2']).dt.days


In [161]:
#list all datatypes in forecast_clean dataframe
forecast_clean.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2617643 entries, 0 to 2617642
Data columns (total 29 columns):
Unnamed: 0                int64
Store                     int64
date_s                    object
Sales                     int64
Customers                 int64
Open                      int64
Promo                     int64
StateHoliday              object
SchoolHoliday             int64
year                      int64
day                       int64
iso_weeknum               int64
weekday                   object
month                     object
seasons                   object
quarter                   int64
mapped_region             object
google_svi                int64
wc_days                   int64
num_asy_app               object
cpi                       float64
tmpc                      float64
relh                      float64
StoreType                 object
Assortment                object
CompetitionDistance       float64
days_since_competition    int64
days_s

In [162]:
#convert datatypes
forecast_clean['Store'] = forecast_clean['Store'].astype(str)
forecast_clean['StateHoliday'] = forecast_clean['StateHoliday'].astype(str)
forecast_clean['SchoolHoliday'] = forecast_clean['SchoolHoliday'].astype(str)
forecast_clean['num_asy_app'] = (forecast_clean['num_asy_app'].apply(lambda x: x.replace(',','').replace('.',''))).astype(float)
#remove date columns since they are captured separately
forecast_clean.drop('date_s', inplace=True, axis = 1)

In [163]:
forecast_clean['num_asy_app'].unique()

array([39825., 37030., 27595., 28720., 33865., 27695., 26875., 20370.,
       22500., 22250., 20100., 18705., 20930., 14655., 12965., 11890.,
       11700., 11655., 14920., 11005., 14425., 15625., 14030., 11545.,
       11365.,  9730.,  8580.,  8735.,  6500.,  6800.,  8370.])

<h3>Linear Regression</h3>
<hr>
<ul>
    <li> One Hot Encoding
    <li> Train, Validation & Test Split
    <li> Feature Selection
        <ul>
            <li> Backward Feature Selection
            <li> Forward Feature Selection
            <li> Stepwise Feature Selection
        </ul>
    <li> Validate model using validation dataset
    <li> Optimize Parameters to get an ideal fit for validation dataset
    <li> Test with test dataset
    <li> Intepret results
</ul>
<br>
<h4>Notes:</h4>
<ul>
    <li> Maynot be an ideal method to forecast a timeseries
    <li> Check from multi-collinearity
    <li> Check for missing or incomplete data
    <li> Check for outliers
    <li> Ensure data splits are made in sequence since this for a time series
</ul>
<br>
<h4>Reference:</h4> 
<ul>
    <li><a href="https://www.theanalysisfactor.com/13-steps-regression-anova/">13 Steps for Regression ANOVA</a>

In [167]:
#creating copy for linear regression
linear_ds = copy.deepcopy(forecast_clean)
#One Hot Encoding
linear_ds = pd.get_dummies(linear_ds)

In [175]:
linear_ds.head()

Unnamed: 0  Sales  Customers  Open  Promo  year  day  iso_weeknum  quarter  \
0           0   5263        555     1      1  2015   31           31        3   
1           1   5263        555     1      1  2015   31           31        3   
2           2   5020        546     1      1  2015   30           31        3   
3           3   5020        546     1      1  2015   30           31        3   
4           4   4782        523     1      1  2015   29           31        3   

   google_svi      ...       mapped_region_Saxony-Anhalt  \
0          51      ...                                 0   
1          51      ...                                 0   
2          51      ...                                 0   
3          51      ...                                 0   
4          51      ...                                 0   

   mapped_region_Schleswig Holstein  mapped_region_Thuringia  StoreType_a  \
0                                 0                        0            0   
1                                 0                        0            0   
2                                 0                        0            0   
3                                 0                        0            0   
4                                 0                        0            0   

   StoreType_b  StoreType_c  StoreType_d  Assortment_a  Assortment_b  \
0            0            1            0             1             0   
1            0            1            0             1             0   
2            0            1            0             1             0   
3            0            1            0             1             0   
4            0            1            0             1             0   

   Assortment_c  
0             0  
1             0  
2             0  
3             0  
4             0  

[5 rows x 1182 columns]

<h3>Train, Validation & Test Split</h3><hr>

In [ ]:
%who

<div style="background-color:#7a1400;height:40px;padding-top:0.1px;">
<h6 align=center style="color:white">
<a href="https://www.linkedin.com/in/avinashpasupulate/" style="color:white">Avinash Pasupulate - Linkedin Profile</a>
</h6> 
</div>